In [2]:
import sys
sys.path.insert(0, '../../Utilities/')
import tensorflow as tf
print(tf.__version__)
import logging
tf.get_logger().setLevel(logging.ERROR)
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
import time
from itertools import product, combinations
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from plotting import newfig, savefig
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import pandas as pd

1.15.5


In [14]:
def load_csv(file_path):
    all_snapshots = []
    lines_per_snapshot = 22341
    lines_of_garbage = 6
    lines_per_section = lines_per_snapshot + lines_of_garbage
    column_names = ['p', 'c', 'w', 'k', 'u', 'v', 'x', 'y']
    snapshot_number = 0
    for chunk in pd.read_csv(file_path, 
                             chunksize=lines_per_section,
                             skip_blank_lines=False,
                             names=column_names,
                             delimiter=','):       
        snapshot_number += 1
        chunk['snapshot'] = None
        data_lines = chunk.iloc[lines_of_garbage:].copy()
        if (len(data_lines)==0):break
        print(f"Loading snapshot {snapshot_number}/100, Length: {len(data_lines)}", end='\r', flush=True)
        data_lines['t'] = snapshot_number
        all_snapshots.append(data_lines)
    all_data = pd.concat(all_snapshots, ignore_index=True)
    return all_data

In [4]:
np.random.seed(1234)
tf.set_random_seed(1234)

In [94]:
class PhysicsInformedNN:
    # Initialize the class
    def __init__(self, x, y, t, wind, leak_x, leak_y, leak_s, u, v, c, k, w, layers):        
        X = np.concatenate([x, y, t, wind, leak_x, leak_y, leak_s], 1)
        
        self.lb = X.min(0)
        self.ub = X.max(0)
                
        self.X = X
        
        self.x = X[:,0:1]
        self.y = X[:,1:2]
        self.t = X[:,2:3]
        self.wind = X[:,3:4]
        self.leak_s = X[:,4:5]
        self.leak_x = X[:,5:6]
        self.leak_y = X[:,6:7]
    
        self.u = u
        self.v = v
        self.c = c
        self.k = k
        self.w = w
        
        self.layers = layers
        
        # Initialize NN
        self.weights, self.biases = self.initialize_NN(layers)        
        
        # Define and Initialize parameters
        #self.rho = tf.Variable([0.0], dtype=tf.float32)
        #self.lambda_2 = tf.Variable([0.0], dtype=tf.float32)
        
        # tf placeholders and graph
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=False))
             
        self.x_tf = tf.placeholder(tf.float32, shape=[None, self.x.shape[1]]) #self.x.shape[1] is 1
        self.y_tf = tf.placeholder(tf.float32, shape=[None, self.y.shape[1]])
        self.t_tf = tf.placeholder(tf.float32, shape=[None, self.t.shape[1]])
        self.wind_tf = tf.placeholder(tf.float32, shape=[None, self.wind.shape[1]])
        self.leak_x_tf = tf.placeholder(tf.float32, shape=[None, self.leak_x.shape[1]])
        self.leak_y_tf = tf.placeholder(tf.float32, shape=[None, self.leak_y.shape[1]])
        self.leak_s_tf = tf.placeholder(tf.float32, shape=[None, self.leak_s.shape[1]])
        
        self.u_tf = tf.placeholder(tf.float32, shape=[None, self.u.shape[1]])
        self.v_tf = tf.placeholder(tf.float32, shape=[None, self.v.shape[1]])
        self.c_tf = tf.placeholder(tf.float32, shape=[None, self.c.shape[1]])
        self.k_tf = tf.placeholder(tf.float32, shape=[None, self.k.shape[1]])
        self.w_tf = tf.placeholder(tf.float32, shape=[None, self.w.shape[1]])
         
        self.u_pred, self.v_pred, self.c_pred, self.k_pred, self.w_pred, self.f_continuity, self.f_convection =\
            self.net_NS(self.x_tf, self.y_tf, self.t_tf, self.wind_tf, self.leak_x_tf, self.leak_y_tf, self.leak_s_tf)

#         self.loss = tf.reduce_sum(tf.square(self.u_tf - self.u_pred)) + \
#                     tf.reduce_sum(tf.square(self.v_tf - self.v_pred)) + \
#                     tf.reduce_sum(tf.square(self.c_tf - self.c_pred)) + \
#                     tf.reduce_sum(tf.square(self.k_tf - self.k_pred)) + \
#                     tf.reduce_sum(tf.square(self.w_tf - self.w_pred)) + \
#                     tf.reduce_sum(tf.square(self.f_continuity)) + \
#                     tf.reduce_sum(tf.square(self.f_convection))
        self.loss = tf.reduce_sum(tf.square(self.u_tf - self.u_pred))
    
    
        self.closs = tf.reduce_sum(tf.square(self.c_tf - self.c_pred)) # not used in trainig
                    
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, 
                                                                method = 'L-BFGS-B', 
                                                                options = {'maxiter': 50000,
                                                                           'maxfun': 50000,
                                                                           'maxcor': 50,
                                                                           'maxls': 50,
                                                                           'ftol' : 1.0 * np.finfo(float).eps}) 
        self.train_op_BFGS = self.optimizer.minimize(self.loss)
        
        self.optimizer_Adam = tf.train.AdamOptimizer()
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)                    
        
        init = tf.global_variables_initializer()
        self.sess.run(init)
        
    def initialize_NN(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            #W = self.xavier_init(size=[layers[l], layers[l+1]])
            W = tf.Variable(tf.zeros([layers[l],layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)        
        return weights, biases
    
    def xavier_init(self, size): #other name: Glorot
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
    
    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1
        # maping all variables to [-1,1]
        H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0  
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y
    
    def net_NS(self, x, y, t, wind, leak_x, leak_y, leak_s):
        # Concatenating all input variables
        input_data = tf.concat([x, y, t, wind, leak_x, leak_y, leak_s], 1)


        u_v_c_k_w = self.neural_net(input_data, self.weights, self.biases)
        u = u_v_c_k_w[:,0:1]
        v = u_v_c_k_w[:,1:2]
        c = u_v_c_k_w[:,2:3]
        k = u_v_c_k_w[:,3:4]
        w = u_v_c_k_w[:,4:5]
    
        # Calculating the required derivatives
        u_x = tf.gradients(u, x)[0]
        v_y = tf.gradients(v, y)[0]
        c_x = tf.gradients(c, x)[0]
        c_y = tf.gradients(c, y)[0]
        c_t = tf.gradients(c, t)[0]
        c_xx = tf.gradients(c_x, x)[0]
        c_yy = tf.gradients(c_y, y)[0]
        
        f_continuity = u_x + v_y
        #f_convection = self.compute_f_convection(u, v, c, k, w, u_x, v_y, c_x, c_y, c_t, c_xx, c_yy)
        f_convection =  u_x + v_y

        return u, v, c, k, w, f_continuity, f_convection 

    def compute_f_convection(self, u, v, c, k, w, u_x, v_y, c_x, c_y, c_t, c_xx, c_yy):
        transient_term = c_t
        convective_term =  u*c_x + v*c_y # assuming c * (u_x + v_y) is zero or continuity is satisfied
        diffusive_term = (1.7e-05 + (2.08e-5*k / w) / 0.803) * (c_xx + c_yy)
        summation = transient_term + convective_term + diffusive_term     
        return summation
    
    def callback(self, loss):
        print('Loss: %.3e, Concentraion loss: %.3e' % (loss))
      
    def train(self, nIter): 
        tf_dict = {self.x_tf: self.x, self.y_tf: self.y, self.t_tf: self.t, self.wind_tf: self.wind,
                   self.leak_x_tf: self.leak_x, self.leak_y_tf: self.leak_y, self.leak_s_tf: self.leak_s,
                   self.u_tf: self.u, self.v_tf: self.v, self.c_tf: self.c,
                   self.k_tf: self.k, self.w_tf: self.w}
        
        start_time = time.time()
        for it in range(nIter):
            self.sess.run(self.train_op_Adam, tf_dict)
            #self.sess.run(self.train_op_BFGS, tf_dict)
            
            # Print
            if it % 1 == 0:
                elapsed = time.time() - start_time
                loss_value = self.sess.run(self.loss, tf_dict)
                closs_value = self.sess.run(self.closs, tf_dict)

                #lambda_1_value = self.sess.run(self.lambda_1)
                #lambda_2_value = self.sess.run(self.lambda_2)
                print('It: %d, Loss: %.3e, cLoss: %.3e, Time: %.2f' % 
                      (it, loss_value,closs_value, elapsed))
                start_time = time.time()
           
        self.optimizer.minimize(self.sess,
                                feed_dict = tf_dict,
                                fetches = [self.loss],
                                loss_callback = self.callback)
            
#     def predict(self, x_star, y_star, t_star, wind_star, leak_x_star, leak_y_star, leak_s_star):
#         tf_dict = {self.x_tf: x_star, self.y_tf: y_star, self.t_tf: t_star, self.wind_tf: wind_star,
#                    self.leak_x_tf: leak_x_star, self.leak_y_tf: leak_y_star, self.leak_s_tf: leak_s_tf}
#         u_star = self.sess.run(self.u_pred, tf_dict)
#         v_star = self.sess.run(self.v_pred, tf_dict)
#         c_star = self.sess.run(self.p_pred, tf_dict)
#         return u_star, v_star, c_star

In [95]:
file_path="/input/data/nmh.csv"
#data = load_csv(file_path).astype('float')

In [96]:
if __name__ == "__main__": 
      
    #N_train = 5000
    N_train = 2
    
    #layers = [7, 20, 20, 20, 20, 20, 20, 20, 20, 5]
    layers = [7, 5]
    
    x = np.array(data['x']).flatten()[:,None]
    y = np.array(data['y']).flatten()[:,None]
    t = np.array(data['t']).flatten()[:,None]
    u = np.array(data['u']).flatten()[:,None]
    v = np.array(data['v']).flatten()[:,None]
    c = np.array(data['c']).flatten()[:,None]
    k = np.array(data['k']).flatten()[:,None]
    w = np.array(data['w']).flatten()[:,None]
    wind = np.full_like(x, 9) # wind speed
    leak_x = np.full_like(x, 0)
    leak_y = np.full_like(x, 1)
    leak_s = np.full_like(x, 1) # leak speed

    # Training Data    
    idx = np.random.choice(len(x), N_train, replace=False)
    x_train = x[idx,:]
    y_train = y[idx,:]
    t_train = t[idx,:]
    u_train = u[idx,:]
    v_train = v[idx,:]
    c_train = c[idx,:]
    k_train = k[idx,:]
    w_train = w[idx,:]
    wind_train = wind[idx,:]
    leak_x_train = leak_x[idx,:]
    leak_y_train = leak_y[idx,:]
    leak_s_train = leak_s[idx,:]

    # Training
    model = PhysicsInformedNN(x_train, y_train, t_train, wind_train,
                              leak_x_train, leak_y_train, leak_s_train,
                              u_train, v_train, c_train, k_train, w_train, layers)
    
    model.train(21)

2023-10-14 02:48:33.685516: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1068] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-14 02:48:33.685612: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 0 with properties: 
name: Quadro RTX 6000 major: 7 minor: 5 memoryClockRate(GHz): 1.77
pciBusID: 0000:17:00.0
2023-10-14 02:48:33.686174: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1068] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-14 02:48:33.686195: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 1 with properties: 
name: Quadro RTX 6000 major: 7 minor: 5 memoryClockRate(GHz): 1.77
pciBusID: 0000:73:00.0
2023-10-14 02:48:33.686824: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1068] could not open file to read NUMA node: /sys

OperatorNotAllowedInGraphError: using a `tf.Tensor` as a Python `bool` is not allowed in Graph execution. Use Eager execution or decorate this function with @tf.function.

In [ ]:
    # Test Data
    snap = np.array([100])
    x_star = X_star[:,0:1]
    y_star = X_star[:,1:2]
    t_star = TT[:,snap]
    
    u_star = U_star[:,0,snap]
    v_star = U_star[:,1,snap]
    p_star = P_star[:,snap]
    
    # Prediction
    u_pred, v_pred, p_pred = model.predict(x_star, y_star, t_star, wind_star, leak_star, angle_star)
    lambda_1_value = model.sess.run(model.lambda_1)
    lambda_2_value = model.sess.run(model.lambda_2)
    
    # Error
    error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
    error_v = np.linalg.norm(v_star-v_pred,2)/np.linalg.norm(v_star,2)
    error_p = np.linalg.norm(p_star-p_pred,2)/np.linalg.norm(p_star,2)

    error_lambda_1 = np.abs(lambda_1_value - 1.0)*100
    error_lambda_2 = np.abs(lambda_2_value - 0.01)/0.01 * 100
    
    print('Error u: %e' % (error_u))    
    print('Error v: %e' % (error_v))    
    print('Error p: %e' % (error_p))    
    print('Error l1: %.5f%%' % (error_lambda_1))                             
    print('Error l2: %.5f%%' % (error_lambda_2))                  
    
    # Plot Results
#    plot_solution(X_star, u_pred, 1)
#    plot_solution(X_star, v_pred, 2)
#    plot_solution(X_star, p_pred, 3)    
#    plot_solution(X_star, p_star, 4)
#    plot_solution(X_star, p_star - p_pred, 5)
    
    # Predict for plotting
    lb = X_star.min(0)
    ub = X_star.max(0)
    nn = 200
    x = np.linspace(lb[0], ub[0], nn)
    y = np.linspace(lb[1], ub[1], nn)
    X, Y = np.meshgrid(x,y)
    
    UU_star = griddata(X_star, u_pred.flatten(), (X, Y), method='cubic')
    VV_star = griddata(X_star, v_pred.flatten(), (X, Y), method='cubic')
    PP_star = griddata(X_star, p_pred.flatten(), (X, Y), method='cubic')
    P_exact = griddata(X_star, p_star.flatten(), (X, Y), method='cubic')

In [93]:
def plot_solution(X_star, u_star, index):
    
    lb = X_star.min(0)
    ub = X_star.max(0)
    nn = 200
    x = np.linspace(lb[0], ub[0], nn)
    y = np.linspace(lb[1], ub[1], nn)
    X, Y = np.meshgrid(x,y)
    
    U_star = griddata(X_star, u_star.flatten(), (X, Y), method='cubic')
    
    plt.figure(index)
    plt.pcolor(X,Y,U_star, cmap = 'jet')
    plt.colorbar()
    
def axisEqual3D(ax):
    extents = np.array([getattr(ax, 'get_{}lim'.format(dim))() for dim in 'xyz'])
    sz = extents[:,1] - extents[:,0]
    centers = np.mean(extents, axis=1)
    maxsize = max(abs(sz))
    r = maxsize/4
    for ctr, dim in zip(centers, 'xyz'):
        getattr(ax, 'set_{}lim'.format(dim))(ctr - r, ctr + r)